# Build Net

In [1]:
import os
import random
import string
from math import comb
from bni_netica.bni_netica import Net
from benchmarking.data_generator import build_random_bn
from bn_helpers.support_tools import get_nets, printNet

Loading Netica


In [2]:
output_path = "./nets/outputs/"

net3 = build_random_bn(
    n_nodes=12,
    name="SkewBN",
    cpt_mode="random",
    dirichlet_alpha=0.3,     # spiky rows
    avg_edges_per_node=1.4,
    max_in_degree=2,
    sprinkle_motifs=5,
    # save_path=output_path
)

printNet(net3)

A -> ['F']
E -> ['L', 'F']
F -> ['D']
D -> []
L -> []
I -> ['C', 'H']
B -> ['K', 'J']
J -> ['C']
C -> []
G -> ['K']
K -> ['H']
H -> []


## Test the boundary

In [3]:
net2 = build_random_bn(
    n_nodes=1000,
    name="MaxRandomBN",
    cpt_mode="random",
    avg_edges_per_node=2,
    max_in_degree=2,
    sprinkle_motifs=5,
    # save_path=output_path
)

# printNet(net2)


# Build Dataset

In [4]:
import pandas as pd
import tempfile
import os

def net_to_dne_string(net):
    """Export a Netica Net to its DNE-format string by writing to temp file."""
    with tempfile.NamedTemporaryFile(suffix=".dne", delete=False) as tmp:
        tmp_path = tmp.name
    try:
        net.write(tmp_path)
        with open(tmp_path, "r") as f:
            return f.read()
    finally:
        if os.path.exists(tmp_path):
            os.remove(tmp_path)

def dne_string_to_net(dne_str):
    """Reconstruct a Netica Net from a DNE-format string."""
    from bni_netica.bni_netica import Net
    with tempfile.NamedTemporaryFile(suffix=".dne", delete=False, mode="w") as tmp:
        tmp.write(dne_str)
        tmp_path = tmp.name
    try:
        net = Net(tmp_path)  # assuming .read(path) works
        net.compile()
        return net
    finally:
        if os.path.exists(tmp_path):
            os.remove(tmp_path)

def save_nets_to_parquet(nets, path="nets.parquet"):
    records = [{"name": net.name(), "dne": net_to_dne_string(net)} for net in nets]
    df = pd.DataFrame(records)
    df.to_parquet(path, index=False)
    print(f"Saved {len(nets)} nets to {path}")

def load_nets_from_parquet(path="nets.parquet"):
    df = pd.read_parquet(path)
    nets = []
    for _, row in df.iterrows():
        net = dne_string_to_net(row["dne"])
        nets.append(net)
    print(f"Loaded {len(nets)} nets from {path}")
    return nets

In [5]:
# import all nets from the collection
nets_collection = get_nets()

# print all nets
for net in nets_collection:
    printNet(net)
    print()

A -> ['B', 'C']
B -> ['D']
C -> ['D', 'E']
D -> []
E -> []

VisitAsia -> ['Tuberculosis']
Tuberculosis -> ['TbOrCa']
Smoking -> ['Cancer', 'Bronchitis']
Cancer -> ['TbOrCa']
TbOrCa -> ['XRay', 'Dyspnea']
XRay -> []
Bronchitis -> ['Dyspnea']
Dyspnea -> []

Class -> ['Obsv', 'Result']
Obsv -> ['Classifier']
Classifier -> ['Result']
Result -> []

Sex -> ['Smoking_status', 'LVH', 'Coronary_artery_disease']
Smoking_status -> ['Coronary_artery_disease']
Systolic_Blood_Pressure -> ['LVH', 'Coronary_artery_disease']
Age -> ['LVH', 'Coronary_artery_disease']
LVH -> ['Coronary_artery_disease']
Diabetes_mellitus -> ['Coronary_artery_disease']
Total_chol -> ['Coronary_artery_disease']
HDL_Status -> ['Coronary_artery_disease']
Coronary_artery_disease -> []

Tampering -> ['Alarm']
Fire -> ['Alarm', 'Smoke']
Alarm -> ['Leaving']
Leaving -> ['Report']
Smoke -> []
Report -> []

P1 -> ['C', 'Color_P1']
P2 -> ['C', 'Color_P2']
C -> ['Color_C']
Color_P1 -> []
Color_C -> []
Color_P2 -> []

Ecstazine -> ['N

In [ ]:
# Generate 500 random nets
data_output = "./benchmarking/data"
nets = nets_collection + \
[ 
    build_random_bn(n_nodes=i,
        cpt_mode="random",
        avg_edges_per_node=2,
        max_in_degree=2,
        sprinkle_motifs=5
    )
    for i in range(3,800)
]

# Save them
save_nets_to_parquet(nets, os.path.join(data_output, "nets_dataset_800.parquet"))

In [ ]:
LOAD_NETS = False
# Load them back
if LOAD_NETS:
    loaded_nets = load_nets_from_parquet(os.path.join(data_output, "nets_dataset.parquet"))
